In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib


In [ ]:
# Load and preprocess the data
data = pd.read_csv("D:\\finalProject\\landmarks_all.csv")
X = data.drop(columns=["label"]).values
y = data["label"]

In [ ]:
data.head()

In [ ]:
# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
# Remove classes with fewer than 2 samples
class_counts = np.bincount(y_encoded)
valid_classes = np.where(class_counts > 1)[0]
valid_indices = np.isin(y_encoded, valid_classes)
X_valid = X[valid_indices]
y_valid = y_encoded[valid_indices]

# Stratified split for balanced training and validation sets
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, val_index in split.split(X_valid, y_valid):
    X_train, X_val = X_valid[train_index], X_valid[val_index]
    y_train, y_val = y_valid[train_index], y_valid[val_index]

In [ ]:
# Reshape input for LSTM if necessary
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

In [ ]:
# Save the label encoder for future use
joblib.dump(label_encoder, r"D:\\finalProject\\training the data\\label_encoder.pkl")

In [ ]:

# Build the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(64))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [ ]:
from tensorflow.keras.utils import to_categorical

# Assuming `num_classes` is the total number of classes (9027 in this case)
y_train = to_categorical(y_train, num_classes=len(label_encoder.classes_))
y_val = to_categorical(y_val, num_classes=len(label_encoder.classes_))


In [ ]:
# model 1 75%
#model = Sequential()
#model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
#model.add(LSTM(64))
#model.add(Dense(len(label_encoder.classes_), activation='softmax'))
## model 2 ####0/76%
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam

initial_learning_rate = 0.001
lr_schedule = ExponentialDecay(
    initial_learning_rate, 
    decay_steps=1000,  # adjust based on your data
    decay_rate=0.96,  # adjust the decay rate as needed
    staircase=True  # if True, learning rate decays at discrete intervals
)
optimizer = Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])



In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=40, batch_size=64)



In [ ]:
# Save the model
model.save("D:\\finalProject\\training the data\\asl_sign_model3.h5")

print("Model trained and saved.")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training loss', 'val_loss'])